## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [3]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [ ]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 一层全连接层
        inner_dim = 128
        self.W1 = tf.Variable(shape=[784, inner_dim], dtype=tf.float32, 
        initial_value=tf.random.normal(shape=[784, inner_dim], stddev=0.1))
        self.b1 = tf.Variable(shape=[inner_dim], dtype=tf.float32, initial_value=tf.random.normal(shape=[inner_dim], stddev=0.1))

        self.W2 = tf.Variable(shape=[inner_dim, 10], dtype=tf.float32,
        initial_value=tf.random.normal(shape=[inner_dim, 10], stddev=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.random.normal(shape=[10], stddev=0.1))


    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        x = tf.matmul(x, self.W1) + self.b1
        x = tf.nn.relu(x)
        logits = tf.matmul(x, self.W2) + self.b2

        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [ ]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.005*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [ ]:
train_data, test_data = mnist_dataset()
for epoch in range(2000):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 0.45088643 ; accuracy 0.8750833
epoch 1 : loss 0.45085832 ; accuracy 0.87511665
epoch 2 : loss 0.45083028 ; accuracy 0.87511665
epoch 3 : loss 0.45080215 ; accuracy 0.87513334
epoch 4 : loss 0.4507741 ; accuracy 0.87515
epoch 5 : loss 0.45074603 ; accuracy 0.87515
epoch 6 : loss 0.4507179 ; accuracy 0.87515
epoch 7 : loss 0.45068985 ; accuracy 0.87516665
epoch 8 : loss 0.45066175 ; accuracy 0.87515
epoch 9 : loss 0.4506338 ; accuracy 0.87515
epoch 10 : loss 0.45060566 ; accuracy 0.87515
epoch 11 : loss 0.45057774 ; accuracy 0.87515
epoch 12 : loss 0.45054966 ; accuracy 0.87516665
epoch 13 : loss 0.4505217 ; accuracy 0.87518334
epoch 14 : loss 0.45049363 ; accuracy 0.8752
epoch 15 : loss 0.45046565 ; accuracy 0.8752
epoch 16 : loss 0.4504377 ; accuracy 0.8752
epoch 17 : loss 0.4504097 ; accuracy 0.87518334
epoch 18 : loss 0.4503817 ; accuracy 0.87518334
epoch 19 : loss 0.4503537 ; accuracy 0.8752
epoch 20 : loss 0.45032585 ; accuracy 0.87521666
epoch 21 : loss 0.45029786 